In [85]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# auto-reload library if developing library functionalities
%reload_ext autoreload
%autoreload 2
# Add the old-can-decoder-c56d4fe6 directory to sys.path
sys.path.append(os.path.abspath('old-can-decoder-c56d4fe6'))

# import library
from PandaCANDecoder.decoder import Decoder as OldDecoder
from can_decoder.decoder import Decoder

In [86]:
# Import df and divide by networks

df_all = pd.read_csv('data/SampleRoadAcceleration_ALL.csv')

# decoder_FD3 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 01'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_FD2 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 07'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_FD11 = Decoder(
#     df_all[df_all['network'] == 'DW CAN 02'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_IHS = Decoder(
#     df_all[df_all['network'] == 'DW CAN 05'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_IHSAT = Decoder(
#     df_all[df_all['network'] == 'DW CAN 06'].drop(columns=['network']).reset_index(drop=True)
#     )

# decoder_C = Decoder(
#     df_all[df_all['network'] == 'DW CAN 08'].drop(columns=['network']).reset_index(drop=True)
#     )

# NETWORK: CAN FD3

## Initial

In [87]:
decoder_FD3 = Decoder(
    df_all[df_all['network'] == 'DW CAN 01'].drop(columns=['network']).reset_index(drop=True)
    )

In [88]:
## Parameters, Load Messages, Dynamic Filtering, 
byte_filters = {
    "18DA10F1": ["1", "2", "3", "4",'5'],
    "18DAF110": ["1", "2", "3", "4",'5'],
    "18DAF218": ["1", "2", "3", "4",'5'],
    "18DA18F2": ["1", "2", "3", "4",'5'],
}

decoder_FD3.generate_msgs(byte_filters=byte_filters)

## Dynamic filtering: if byte 1 is 0 we should discard byte 5
for msg in decoder_FD3.msgs:
    if msg.msg_byte_filter is not None:
        if msg.msg_byte_filter['1'] != 0:
            msg.msg_byte_filter.pop('5', None)
            # Remove duplicates

# Remove duplicate messages with same msg_id and identical msg_byte_filter after discarding byte 5
unique_msgs = []
seen = set()
for msg in decoder_FD3.msgs:
    key = (msg.msg_id, tuple(sorted(msg.msg_byte_filter.items())) if msg.msg_byte_filter else None)
    if key not in seen:
        unique_msgs.append(msg)
        seen.add(key)
decoder_FD3.msgs = unique_msgs



Generated 910 messages


In [89]:
## TS data and signal decoding

decoder_FD3.generate_msg_ts_data(rewrite=True)

decoder_FD3.calculate_signals(
        tokenization_method='conditional_bit_flip',
        signedness_method='msb_classifier',
        alpha1=0.01,
        alpha2=0.5, #0.5,
        gamma1=0.2)

## Signal Setup

In [90]:
from can_decoder.message import Message
from can_decoder.signal import Signal

# [m for m in decoder_FD3.msgs if m.msg_id in ['18DAF218', '18DAF110', '18DAF228']]
sorted([
    (f"{m.msg_id}", 
     [hex(int(f)).rjust(4) for f in m.msg_byte_filter.values()])
    for m in decoder_FD3.msgs if m.msg_id in ['18DAF110']
], key=lambda el: int(el[1][-2], 16)*16 + int(el[1][-1], 16))



[('18DAF110', [' 0x5', '0x62', ' 0x1', ' 0x6']),
 ('18DAF110', [' 0x5', '0x62', ' 0x1', ' 0x9']),
 ('18DAF110', [' 0x4', '0x62', ' 0x1', '0x1d']),
 ('18DAF110', [' 0x4', '0x62', ' 0x1', '0x20']),
 ('18DAF110', [' 0x4', '0x62', ' 0x1', '0x23']),
 ('18DAF110', [' 0x4', '0x62', ' 0x1', '0x27']),
 ('18DAF110', [' 0x5', '0x62', ' 0x1', '0x2b']),
 ('18DAF110', [' 0x4', '0x62', ' 0x2', '0x27']),
 ('18DAF110', [' 0x5', '0x62', ' 0x2', '0x28']),
 ('18DAF110', [' 0x4', '0x62', ' 0x2', '0x2a']),
 ('18DAF110', [' 0x7', '0x62', ' 0x2', '0x33']),
 ('18DAF110', [' 0x7', '0x62', ' 0x2', '0x34']),
 ('18DAF110', [' 0x7', '0x62', ' 0x2', '0x35']),
 ('18DAF110', [' 0x7', '0x62', ' 0x2', '0x36']),
 ('18DAF110', [' 0x7', '0x62', ' 0x2', '0x37']),
 ('18DAF110', [' 0x5', '0x62', ' 0x2', '0x55']),
 ('18DAF110', [' 0x5', '0x62', ' 0x2', '0x56']),
 ('18DAF110', [' 0x5', '0x62', ' 0x2', '0x57']),
 ('18DAF110', [' 0x5', '0x62', ' 0x2', '0x58']),
 ('18DAF110', [' 0x5', '0x62', ' 0x1', '0x74']),
 ('18DAF110', [' 0x5

In [95]:
m = decoder_FD3.get_message('18DAF110', [5, 98, 0x1, 0xD5])
m.signals = [
    Signal(
        start_bit = 39,
        length = 16,
        byte_order='be',
        classification="ts",
        msg=m,
        name='PCM_Engine_Speed_RPM',
    ),
]

In [96]:
m.plot()

In [100]:
candidates = decoder_FD3.find_signal_match(
    m.get_signal('PCM_Engine_Speed_RPM'),
    thresh=0.9,
    only_ts=True
)
f = decoder_FD3.plot_signal_matches(
    m.get_signal('PCM_Engine_Speed_RPM'), 
    candidates=candidates,
    return_fig=True)
f.show()

PCM_Engine_Speed_RPM --> S_40B_BE_0           in 40B        , r^2=0.999951
PCM_Engine_Speed_RPM --> S_40B_LE_13          in 40B        , r^2=0.999924
PCM_Engine_Speed_RPM --> S_106_BE_1           in 106        , r^2=0.993198
PCM_Engine_Speed_RPM --> S_106_BE_3           in 106        , r^2=0.999987
PCM_Engine_Speed_RPM --> S_106_LE_31          in 106        , r^2=0.997794
PCM_Engine_Speed_RPM --> S_106_LE_34          in 106        , r^2=0.990037
PCM_Engine_Speed_RPM --> S_114_BE_5           in 114        , r^2=0.999512
PCM_Engine_Speed_RPM --> S_114_LE_54          in 114        , r^2=0.996725
PCM_Engine_Speed_RPM --> S_18DAF218_BE_1      in 18DAF218   , r^2=0.995678
PCM_Engine_Speed_RPM --> S_18DAF218_LE_7      in 18DAF218   , r^2=0.993934
PCM_Engine_Speed_RPM --> S_18DAF218_BE_3      in 18DAF218   , r^2=0.995616
PCM_Engine_Speed_RPM --> S_18DAF218_LE_14     in 18DAF218   , r^2=0.993823
PCM_Engine_Speed_RPM --> S_7A4_BE_1           in 7A4        , r^2=0.993041
PCM_Engine_Speed_RPM --> 